# Lecture 4: Model-free prediction

- estimate the value function of an _unknown_ MDP

__Monte-Carlo Reinforcement Learning__

- MC methods learn directly from episodes of experience
- MC is _model-free_: no knowledge of MDP transitions/rewards
- MC learns from _complete_ episodes, no bootstrapping
- MC uses the simplest possible idea: value = mean return
- caveat: can only apply MC to episodic MDPs:
  - all episodes must terminate

__Monte-Carlo policy evaluation__

- Goal: learn $v_\pi$ from episodes of experience under policy $\pi$
$$
S_1, A_1, R_2, \dots, S_k \sim \pi
$$
- recall that the _return_ is the total discounted reward
$$
G_t = R_{t+1} + \gamma R_{t+2} + \dots + \gamma^{T-1} R_t
$$
- recall that the value function is the expected return:
$$
v_\pi(s) = \mathbb{E}_\pi[G_t \mid S_t =s]
$$
- Monte-Carlo policy evaluation uses _empirical mean_ return instead of _expected_ return

__First-visit Monte-Carlo Policy Evaluation__

- to evaluate state $s$
- the first time-step $t$ that state $s$ is visited in an episode:
  - increment counter $N(s) \leftarrow N(s) + 1$
  - increment total return $S(s) \leftarrow S(s) + G_t$
- value is estimated by mean return $V(s) = S(s) / N(s)$
- by law of large numbers, $V(s) \rightarrow v_\pi(s)$ as $N(s) \rightarrow \infty$

__Every-visit Monte-Carlo policy evaluation__

- to evaluate state $s$:
- every time-step $t$ that state $s$ is visited in an episode:
  - increment counter $N(s) \leftarrow N(s) + 1$
  - increment total return $S(s) \leftarrow S(s) + G_t$
- value is estimated by mean return $V(s) = S(s) / N(s)$
- again, $V(s) \rightarrow v_\pi(s)$ as $N(s) \rightarrow \infty$

In [62]:
import torch
import numpy as np
import math
import random
import matplotlib.pyplot as plt


N = torch.zeros(10, 10, 2)
S = torch.zeros(10, 10, 2)

def create_cards():
    cards = []
    for v in range(13):
        for i in range(4):
            cards.append(v + 1)
#     print('cards', cards)
    return cards

def calc_sum_cards(cards):
    sum = 0
    num_aces = 0
    for card in cards:
        if card > 10:
            card = 10
        if card == 1:
            num_aces += 1
            card = 11
        sum += card
    while sum > 21 and num_aces > 0:
        sum -= 10
        num_aces -= 1
    return sum, num_aces

def cards_to_state(dealer_card, us):
    sum, num_aces = calc_sum_cards(us)
    s = torch.IntTensor([sum - 12, dealer_card - 1, 0])
    if num_aces > 0:
        s[2] = 1
    return s, sum > 21

def apply_action(cards, s, a):
    our_sum = s[0] + 12
#     dealer_sum = s[1] + 1
    if s[1] == 0:
        dealer_cards = [11]
        dealer_aces = 1
    else:
        dealer_cards = [s[1] + 1]
        dealer_aces = 0
    if a == 0:  # stick
        num_aces = 0
        while calc_sum_cards(dealer_cards)[0] < 16:
            dealer_cards.append(cards[-1])
        dealer_sum = calc_sum_cards(dealer_cards)[0]
        if dealer_sum > 21:
            return cards, _, 1, True
        elif dealer_sum == our_sum:
            return cards, _, 0, True
        else:
            return cards, _, -1, True
    else:  # twist
        card = cards[-1]
        cards = cards[:-1]
        useable_ace = s[2] == 1
        if card > 10:
            card = 10
        if card == 1:
            useable_ace = True
            card = 11
        our_sum += card
        if our_sum > 21 and useable_ace:
            our_sum -= 10
            useable_ace = False
        if our_sum > 21:
#             print('bust')
            return cards, _, -1, True
        else:
            s_new = s.clone()
            s_new[0] = our_sum - 12
            s_new[2] = 1 if useable_ace else 0
            return cards, s_new, 0, False
#     return cards, s_new, reward

episode = 0
# while True:
while episode < 100:
#     print('e %s' % episode)
    cards = create_cards()
    random.shuffle(cards)
#     print('cards', cards)
    dealer_card = cards[-1]
    if dealer_card > 10:
        dealer_card = 10
    cards = cards[:-1]
    our_cards = []
    while calc_sum_cards(our_cards)[0] < 12:
        cards = cards[:-1]
        our_cards.append(cards[-1])
    s, done = cards_to_state(dealer_card, our_cards)
    states = []
    rewards = []
    while True:
#         N[s[0], s[1], s[2]] += 1
#         S[s[0], s[1], s[2]] += 
#         print('s', s)
        states.append(s.clone())
        a = np.random.randint(2)
#         print('a', a)
        cards, s_new, reward, done = apply_action(cards, s, a)
#         print('s_new', s_new)
        rewards.append(reward)
        if done:
            break
    G = 0
#     print('len(states)', len(states))
    for i in range(len(states) - 1, -1, -1):
        s = states[i]
        r = rewards[i]
        G += r
#         print('i', i, 's', s, 'r', r, 'G', G)
        N[s[0], s[1], s[2]] += 1
        S[s[0], s[1], s[2]] += G
#     print('G', G)
#     print('N', N)
#     print('S', S)
    episode += 1

# plt.imshow()

G -1
G -1
G -1
G -1
G -1
G -1
G 0
G 0
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G 0
G -1
G 1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G 1
G -1
G -1
G -1
G -1
G -1
G 1
G -1
G -1
G 1
G 0
G -1
G 0
G -1
G -1
G -1
G -1
G -1
G 0
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G 1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G 1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G -1
G 1
G -1
G -1
G -1
G 1
G -1
G -1
G 1
G -1
G -1
G -1
G -1
G -1
G 1
G -1


up to 55:10 approx

In [124]:
import matplotlib.pyplot as plt
import torch
import numpy as np

class Env(object):
    def __init__(self):
        self.env_size = 7

    @property
    def num_actions(self):
        return 2

    @property
    def state_shape(self):
        return [self.env_size]
    
    def reset(self):
        self.s = torch.IntTensor(1)
        self.s[0] = np.random.randint(5) + 1
        return self.s
        
    def act(self, a):
        if a == 0:
            self.s[0] -= 1
        else:
            self.s[0] += 1
        reward = 1 if self.s[0] == self.env_size - 1 else 0
        done = self.s[0] in [0, self.env_size - 1]
        return self.s, reward, done
    
    def render(self):
        res = ['-'] * self.env_size
        res[0] = '*'
        res[6] = '*'
        res[self.s[0]] = 'X'
        print(''.join(res))

class RandomPolicy(object):
    def __init__(self, num_actions):
        self.num_actions = num_actions
    
    def get_action(self, s):
        return np.random.randint(self.num_actions)

def run_episode(env, policy, render=True):
    actions = []
    rewards = []
    states = []
    s = env.reset()
    while True:
        a = policy.get_action(s)
        if render:
            env.render()
        states.append(s.clone())
        s, r, done = env.act(a)
        rewards.append(r)
        actions.append(a)
        if done:
            break
    if render:
        env.render()
    return states, actions, rewards

env = Env()
policy = RandomPolicy(env.num_actions)
print(env.state_shape)

num_episodes = 5
num_episodes = 0
for episode in range(num_episodes):
    print('')
    print('episode %s' % episode)
    states, actions, rewards = run_episode(env, policy)

def get_linear_index(target, index):
    assert len(index.shape) == 1
    linear_index = 0
    D = index.shape[0]
    for d in range(D):
        linear_index = linear_index * target.shape[d] + index[d]
    return linear_index

def tensor_set(target, index, v):
    linear_index = get_linear_index(target, index)
    target.view(-1)[linear_index] = v

def tensor_get(target, index):
    linear_index = get_linear_index(target, index)
    target = target.view(-1)[linear_index]
    return target

def tensor_inc(target, index, v):
    tensor_set(target, index, tensor_get(target, index) + v)


class MonteCarloAllVisits(object):
    def __init__(self, env, policy):
        self.env = env
        self.policy = policy
        self.state_shape = env.state_shape
        self.N = torch.zeros(self.state_shape)
        self.S = torch.zeros(self.state_shape)

    def step(self):
        states, actions, rewards = run_episode(self.env, self.policy, render=False)
        G = 0
        for i in range(len(actions) - 1, -1, -1):
            s = states[i]
            a = actions[i]
            r = rewards[i]
            G += r
            tensor_inc(self.N, s, 1)
            tensor_inc(self.S, s, G)

    @property
    def V(self):
        return self.S / self.N

runner = MonteCarloAllVisits(env, policy)
for it in range(100):
    runner.step()
    if it % 20 == 0:
        print('it', it)
        print('V', runner.V)
print('V', runner.V)


[7]
it 0
V 
nan
nan
nan
nan
  1
  1
nan
[torch.FloatTensor of size 7]

it 20
V 
    nan
 0.2333
 0.4375
 0.5775
 0.6327
 0.7600
    nan
[torch.FloatTensor of size 7]

it 40
V 
    nan
 0.1875
 0.3889
 0.5800
 0.7308
 0.8696
    nan
[torch.FloatTensor of size 7]

it 60
V 
    nan
 0.1884
 0.3740
 0.5588
 0.7411
 0.8824
    nan
[torch.FloatTensor of size 7]

it 80
V 
    nan
 0.2737
 0.4364
 0.5778
 0.7305
 0.8929
    nan
[torch.FloatTensor of size 7]

V 
    nan
 0.2547
 0.4294
 0.5700
 0.7126
 0.8667
    nan
[torch.FloatTensor of size 7]



In [130]:
class MonteCarloFirstVisit(object):
    def __init__(self, env, policy):
        self.env = env
        self.policy = policy
        self.state_shape = env.state_shape
        self.N = torch.zeros(self.state_shape)
        self.S = torch.zeros(self.state_shape)
#         print('self.N', self.N)

    def step(self):
        states, actions, rewards = run_episode(self.env, self.policy, render=False)
        G = 0
#         seen_states = set()
        seen_states = torch.zeros(self.state_shape)
        seen_states.zero_()
        for i in range(len(actions) - 1, -1, -1):
            s = states[i]
            a = actions[i]
            r = rewards[i]
            G += r
#             print('seen_states', seen_states)
#             print('s', s)
            if tensor_get(seen_states, s) == 0:
                tensor_inc(self.N, s, 1)
                tensor_inc(self.S, s, G)
                tensor_set(seen_states, s, 1)

    @property
    def V(self):
        return self.S / self.N

runner = MonteCarloFirstVisit(env, policy)
for it in range(100):
    runner.step()
    if it % 20 == 0:
        print('it', it)
        print('V', runner.V)
print('V', runner.V)

it 0
V 
nan
nan
  1
  1
  1
  1
nan
[torch.FloatTensor of size 7]

it 20
V 
    nan
 0.0769
 0.3333
 0.5000
 0.5714
 0.9000
    nan
[torch.FloatTensor of size 7]

it 40
V 
    nan
 0.0435
 0.2917
 0.4545
 0.6000
 0.8261
    nan
[torch.FloatTensor of size 7]

it 60
V 
    nan
 0.1389
 0.3514
 0.4706
 0.6176
 0.8108
    nan
[torch.FloatTensor of size 7]

it 80
V 
    nan
 0.1064
 0.3191
 0.4545
 0.6341
 0.8478
    nan
[torch.FloatTensor of size 7]

V 
    nan
 0.1053
 0.3051
 0.4643
 0.6471
 0.8750
    nan
[torch.FloatTensor of size 7]



In [151]:
class TD(object):
    def __init__(self, env, policy, alpha=0.1):
        self.env = env
        self.policy = policy
        self.state_shape = env.state_shape
        self._V = torch.zeros(self.state_shape)
        self.alpha = alpha

    def step(self):
        states, actions, rewards = run_episode(self.env, self.policy, render=False)
        is_terminal = True
        s_next = None
        for t in range(len(actions) - 1, -1, -1):
#             print('t', t)
            s = states[t]
            a = actions[t]
            r = rewards[t]
            v_old = tensor_get(self._V, s)
            v_next = 0
            if s_next is not None:
                v_next = tensor_get(self.V, s_next)
            v_new = v_old + self.alpha * (r + v_next - v_old)
#             if s_next is not None:
#                 print('s', s[0], 'a', a, 'r', r, 's_next', s_next[0], 'v_new', v_new)
#             else:
#                 print('s', s[0], 'a', a, 'r', r, 'v_new', v_new)
            tensor_set(self._V, s, v_new)
            is_terminal = False
            s_next = s

    @property
    def V(self):
        return self._V

runner = TD(env, policy)
for it in range(100):
#     if random.randint(0, 2) == 0:
    runner.step()
    if it % 10 == 0:
        print('it', it)
        print('V', runner.V.view(1, -1))
print('V', runner.V.view(1, -1))

it 0
V 
 0.0000  0.0001  0.0016  0.0064  0.0307  0.1030  0.0000
[torch.FloatTensor of size 1x7]

it 10
V 
 0.0000  0.0040  0.0265  0.1264  0.2425  0.4410  0.0000
[torch.FloatTensor of size 1x7]

it 20
V 
 0.0000  0.0123  0.5195  0.9359  1.4297  1.5510  0.0000
[torch.FloatTensor of size 1x7]

it 30
V 
 0.0000  0.7024  3.0198  6.5217  8.6699  5.8119  0.0000
[torch.FloatTensor of size 1x7]

it 40
V 
  0.0000   1.9465  11.4581  19.2049  18.1844  10.5330   0.0000
[torch.FloatTensor of size 1x7]

it 50
V 
  0.0000  64.5294  98.2167  90.3260  65.9835  25.4893   0.0000
[torch.FloatTensor of size 1x7]

it 60
V 
   0.0000  165.3389  331.6011  351.0712  169.0275   65.9930    0.0000
[torch.FloatTensor of size 1x7]

it 70
V 
    0.0000   893.8348  3545.5962  3703.3303  1736.8694   314.3897     0.0000
[torch.FloatTensor of size 1x7]

it 80
V 

Columns 0 to 5 
     0.0000   5013.9551  14255.0811  18314.0117  10492.3408   3617.6785

Columns 6 to 6 
     0.0000
[torch.FloatTensor of size 1x7]

it 90
V 